In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Change directory to your code
%cd /content/drive/MyDrive/addition
%pwd   # verify you’re in the right place
!ls    # should show train.py, 4_operands_addition.txt, etc.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/addition
configuration_files	legacy_code	       result_analysis.py
configurator.py		main_utilities.py      result_analysis_scripts
data			model.py	       results
data_generate.py	__pycache__	       startHere.ipynb
data_generation_script	README.md	       statistical_measurements.py
evaluation.py		result_analysis.ipynb  train.py


# Quick Start Example
## Choose One Task and Start Training

#### Generate Addition Data

In [4]:
!python data_generate.py --task addition --num_operands 2 --experiment_name 2_operands_0_to_999_uniform --train_size 10000 --test_size 3000 --val_size 3000 --train_eval True --sample-size 3000 --generate_reverse True

Running generator for task 'addition', experiment '2_operands_0_to_999_uniform'
Running: /usr/bin/python3 /content/drive/MyDrive/addition/data_generation_script/individual_task_scripts/addition/addition_gen.py --num_operands 2 --output_dir /content/drive/MyDrive/addition/data/2_operands_0_to_999_uniform --train_size 10000 --test_size 3000 --val_size 3000
Wrote 10000 train -> /content/drive/MyDrive/addition/data/2_operands_0_to_999_uniform/train.txt, 3000 test -> /content/drive/MyDrive/addition/data/2_operands_0_to_999_uniform/test.txt, 3000 val -> /content/drive/MyDrive/addition/data/2_operands_0_to_999_uniform/val.txt (num_operands=2).
Generation finished successfully.
Sampling 3000 lines to create train_eval at '/content/drive/MyDrive/addition/data/2_operands_0_to_999_uniform/train_eval.txt'
Running: /usr/bin/python3 /content/drive/MyDrive/addition/data_generation_script/sample.py --input /content/drive/MyDrive/addition/data/2_operands_0_to_999_uniform/train.txt --output /content/dri

#### Generate Multiplication Data

In [ ]:
!python data_generate.py --task multiplication --experiment_name 2_operands_0_to_999_uniform \
        --train_size 10000 --test_size 3000 --val_size 3000 --train_eval True --sample-size 3000 --generate_reverse True

#### Generate Sorting Data

In [ ]:
!python data_generate.py --task sorting --experiment_name 4_operands_sorting_balanced_digit \
        --train_size 1000000 --test_size 10000 --val_size 10000 --train_eval True --sample-size 10000

## Let's Start Training!

#### The .txt file is the configuration file

In [ ]:
!python train.py 2_operands_addition_reversed.txt

# Other Commands (May not need to use them)

## 4 Operands

### Plain

In [ ]:
%cat 4_operands_addition_plain.txt

In [ ]:
!python train.py 4_operands_addition_plain.txt

## 2 Operands

In [ ]:
!python train.py 2_operands_addition_plain.txt

In [ ]:
!python train.py 2_operands_addition_reversed.txt

# Subtraction

In [ ]:
%cat train.py

In [ ]:
!python train.py 2_operands_subtraction_reversed.txt

## 4 Operand Addition Scratchpad

In [ ]:
%cat configuration_files/4_operands_addition_scratchpad.txt

In [ ]:
%cat evaluation.py

In [ ]:
!python train.py 4_operands_addition_scratchpad.txt

## 4 Operand Max

In [ ]:
%cat configuration_files/4_operands_max.txt

In [ ]:
%cat train.py

In [ ]:
%cat model.py

In [ ]:
!python train.py 4_operands_max.txt

## 4 Operand Sorting

In [ ]:
!python train.py 4_operands_sorting.txt

## Addition OOD

In [ ]:
!python train.py addition_OOD_2_operand.txt --batch slicing

In [ ]:
!python train_slicing_batch.py addition_OOD.txt